# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#cf): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours): Number of crew or vessel hours spent at sea
 - [Number of Tows](#tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-costs): Total fixed costs
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [PySAM-Powered Results](#pysam): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 96.0%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.98,0.98,0.94,0.97,0.98,0.98,0.96,0.98,0.96,0.97,0.88,0.97,0.90,0.95,0.96,0.93,0.98,0.98,0.86,0.93,0.98,0.98,0.98,0.98,0.92,0.97,0.93,0.98,0.98,0.92,0.88,0.98,0.98,0.92,0.99,0.98,0.86,0.96,0.96,0.98,0.96,0.98,0.93,0.98,0.98,0.97,0.92,0.98,0.94,0.93,0.98,0.98,0.98,0.97,0.91,0.98,0.97,0.98,0.98,0.98,0.98,0.95,0.97,0.98,0.98,0.94,0.94,0.98,0.98,0.98,0.93,0.97,0.99,0.97,0.98,0.94,0.97,0.98,0.98,0.97


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.97
2004,0.96
2005,0.94
2006,0.96
2007,0.94
2008,0.97
2009,0.96
2010,0.97
2011,0.97


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.96
3,0.96
4,0.96
5,0.95
6,0.96
7,0.97
8,0.96
9,0.95


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.99
     3          0.99
     4          0.98
     5          0.97
     6          0.97
     7          0.97
     8          0.97
     9          0.97
     10         0.92
     11         0.93
     12         0.94
2004 1          0.96
     2          0.95
     3          0.95
     4          0.94
     5          0.94
     6          0.96
     7          0.98
     8          0.97
     9          0.97
     10         0.96
     11         0.95
     12         0.96

<a id="cf"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.47,0.45,0.46,0.47,0.47,0.46,0.47,0.46,0.46,0.42,0.46,0.43,0.45,0.46,0.44,0.47,0.47,0.41,0.44,0.47,0.47,0.47,0.47,0.44,0.46,0.44,0.46,0.47,0.45,0.42,0.47,0.47,0.44,0.47,0.46,0.41,0.46,0.46,0.47,0.46,0.47,0.44,0.47,0.46,0.47,0.44,0.47,0.45,0.44,0.47,0.47,0.47,0.47,0.44,0.47,0.46,0.47,0.47,0.47,0.47,0.44,0.46,0.47,0.47,0.45,0.44,0.47,0.47,0.47,0.44,0.46,0.47,0.46,0.47,0.44,0.46,0.47,0.47,0.46


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.45
2005,0.46
2006,0.45
2007,0.48
2008,0.51
2009,0.46
2010,0.41
2011,0.49


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.50
3,0.50
4,0.41
5,0.38
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.37
     3          0.38
     4          0.53
     5          0.34
     6          0.30
     7          0.29
     8          0.29
     9          0.27
     10         0.40
     11         0.52
     12         0.53
2004 1          0.54
     2          0.51
     3          0.50
     4          0.38
     5          0.28
     6          0.38
     7          0.27
     8          0.41
     9          0.57
     10         0.55
     11         0.47
     12         0.48

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 99%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.99
2004,0.98
2005,1.03
2006,0.99
2007,1.01
2008,0.98
2009,1.02
2010,1.00
2011,0.99


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.93
2,1.02
3,1.03
4,1.05
5,1.01
6,1.00
7,0.99
8,1.01
9,1.01


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

Completion Rate
year month                 
2003 1                 0.92
     2                 1.02
     3                 1.00
     4                 1.01
     5                 0.99
     6                 1.04
     7                 1.01
     8                 0.96
     9                 1.00
     10                0.94
     11                1.01
     12                0.94
2004 1                 0.51
     2                 1.11
     3                 1.15
     4                 1.25
     5                 1.18
     6                 0.96
     7                 1.15
     8                 1.06
     9                 0.93
     10                1.02
     11                0.81
     12                0.89

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $429,359.77/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,726,004.97","82,140,559.03"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"8,035,484.38"
2004,"6,936,576.39"
2005,"7,732,816.44"
2006,"18,661,652.50"
2007,"19,789,711.97"
2008,"7,037,529.51"
2009,"7,727,036.18"
2010,"7,409,110.26"
2011,"12,152,076.94"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,010.42","543,010.42","543,010.42","288,574.05","4,929,230.90"
2,"496,125.00","496,125.00","495,250.00",0.00,"2,673,128.47"
3,"541,989.58","541,989.58","542,864.58","120,911.47","11,153,998.26"
4,"524,635.42","524,635.42","524,635.42","311,029.23","4,165,625.00"
5,"542,500.00","542,500.00","542,500.00","359,391.16","7,402,456.60"
6,"525,000.00","525,000.00","525,000.00","113,802.08","10,772,538.19"
7,"542,500.00","542,500.00","542,500.00","71,767.11","3,182,812.50"
8,"542,500.00","542,500.00","542,500.00","26,916.67",0.00
9,"525,000.00","525,000.00","525,000.00","109,059.23","3,991,368.06"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
     6                   52,500.00                  0.00               0.00  
     7                   54,250.00                  0.00               0.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00       1,509,890.62  
     12                  54,250.00                  0.00       4,607,812.50  
2004 1                   54,250.00                  0.00               0.00  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       1,265,076.39  
     6                   52,500.00                  0.00       3,750,000.00  
     7                   54,250.00                  0.00               0.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

<a id="utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.91,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.00,0.97
2004,1.00,1.00,1.00,0.00,0.94
2005,1.00,1.00,1.00,0.93,0.94
2006,1.00,1.00,1.00,0.94,0.97
2007,1.00,1.00,1.00,0.93,0.99
2008,1.00,1.00,1.00,0.67,0.94
2009,1.00,1.00,1.00,0.91,0.97
2010,1.00,1.00,1.00,0.95,0.97
2011,1.00,1.00,1.00,0.83,0.94


<a id="vessel-crew-hours"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"121,473.35"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 2,Crew Transfer Vessel 1,Crew Transfer Vessel 3,Heavy Lift Vessel,Field Support Vessel
0,2003,"7,675.83","1,991.75","1,895.50","1,735.50","2,053.08",0.00
1,2004,"13,637.75","3,939.00","4,001.50","3,950.25","1,747.00",0.00
2,2005,"13,955.94","3,435.75","3,418.25","3,413.75","1,970.69","1,717.50"
3,2006,"14,685.05","2,871.75","2,756.00","2,863.00","5,593.30",601.00
4,2007,"15,411.20","2,809.25","2,803.50","2,813.00","5,743.45","1,242.00"
5,2008,"11,298.78","2,982.50","2,886.25","3,128.00","1,798.03",504.00
6,2009,"11,102.04","2,803.75","2,894.25","2,792.00","2,012.04",600.00
7,2010,"11,684.50","2,665.25","2,732.50","2,671.75","1,878.00","1,737.00"
8,2011,"12,422.89","2,664.00","2,548.75","2,516.75","3,589.39","1,104.00"
9,2012,"9,599.36","2,692.00","2,436.50","2,529.50","1,941.36",0.00


<a id="tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

<a id="equipment-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,342,466.15"
Repair,"20,253,527.78"
Crew Transfer,"1,127,432.30"
Site Travel,0.00
Mobilization,"7,500,000.00"
Weather Delay,"35,992,122.41"
No Requests,"15,412,563.35"
Not in Shift,"20,418,233.28"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,342,466.15","2,342,466.15"
Repair,0.00,0.00,0.00,"20,253,527.78","20,253,527.78"
Crew Transfer,0.00,0.00,0.00,"1,127,432.30","1,127,432.30"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"7,500,000.00","7,500,000.00"
Weather Delay,0.00,0.00,0.00,"35,992,122.41","35,992,122.41"
No Requests,0.00,0.00,0.00,"15,412,563.35","15,412,563.35"
Not in Shift,0.00,0.00,0.00,"20,418,233.28","20,418,233.28"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      58,661.46
     Repair        1,560,710.94
     Crew Transfer    82,276.04
     Site Travel           0.00
     Mobilization    500,000.00
     Weather Delay 3,442,984.38
     No Requests   1,244,359.38
     Not in Shift  1,146,492.19
2004 Maintenance     442,895.83
     Repair        1,563,338.54

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                    
1     Maintenance     208,778.65
      Repair        1,653,445.31
      Crew Transfer    93,296.88
      Site Travel           0.00
      Mobilization    500,000.00
      Weather Delay 2,318,710.07
      No Requests     714,488.11
      Not in Shift  1,358,117.19
2     Maintenance     201,778.65
      Repair          561,159.72

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          20,416.67
           Crew Transfer    2,588.54
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   22,458.33
           No Requests    111,088.54
           Not in Shift     7,729.17
     2     Maintenance          0.00
           Repair          21,492.19
           Crew Transfer    2,406.25
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   15,093.75
           No Requests    100,989.58
           Not in Shift     7,018.23
     3     Maintenance          0.00
           Repair          23,041.67
           Crew Transfer    2,697.92
           Site Travel          0.00

<a id="component-costs"></a>
## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"116,537,093.14"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   56,282,166.92   
          maintenance            0.00              0.00    2,342,466.15   
          repair                 0.00              0.00   20,253,527.78   

                         total_cost  
component action                     
turbine   delay       56,282,166.92  
          maintenance  2,342,466.15  
          repair      20,253,527.78

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"2,208,500.00",0.00,"6,289,593.75","8,498,093.75"
2004,turbine,"3,655,500.00",0.00,"5,579,864.58","9,235,364.58"
2005,turbine,"4,194,500.00",0.00,"6,615,332.12","10,809,832.12"
2006,turbine,"4,354,500.00",0.00,"12,683,936.63","17,038,436.63"
2007,turbine,"6,828,000.00",0.00,"17,341,342.89","24,169,342.89"
2008,turbine,"2,558,000.00",0.00,"4,403,961.81","6,961,961.81"
2009,turbine,"3,201,500.00",0.00,"6,199,726.28","9,401,226.28"
2010,turbine,"2,926,000.00",0.00,"5,850,459.20","8,776,459.20"
2011,turbine,"3,915,500.00",0.00,"8,942,915.80","12,858,415.80"


In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"2,770,500.00",0.00,"5,618,191.84","8,388,691.84"
2,turbine,"1,617,500.00",0.00,"3,178,959.20","4,796,459.20"
3,turbine,"3,069,000.00",0.00,"7,909,355.92","10,978,355.92"
4,turbine,"3,095,000.00",0.00,"4,817,769.68","7,912,769.68"
5,turbine,"3,498,000.00",0.00,"6,990,513.89","10,488,513.89"
6,turbine,"5,264,000.00",0.00,"10,543,927.95","15,807,927.95"
7,turbine,"2,559,500.00",0.00,"4,088,429.69","6,647,929.69"
8,turbine,"1,369,000.00",0.00,"805,382.81","2,174,382.81"
9,turbine,"1,758,500.00",0.00,"3,366,968.40","5,125,468.40"


In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         16,000.00              0.00       51,661.46   
     2     turbine         42,500.00              0.00       46,010.42   
     3     turbine         54,000.00              0.00       43,677.08   
     4     turbine         63,000.00              0.00       44,916.67   
     5     turbine         66,000.00              0.00       47,377.60   
     6     turbine         69,500.00              0.00       26,250.00   
     7     turbine         76,500.00              0.00       43,421.88   
     8     turbine         32,500.00              0.00       30,369.79   
     9     turbine         29,500.00              0.00       35,838.54   
     10    turbine        128,000.00              0.00      111,453.13   
     11    turbine        141,500.00              0.00    1,083,773.44   
     12    turbine      1,489,500.00              0.00    4,724,843.75   
2004 1     turbine        140,500.00              0.00      161,765.63   
     2     turbine        237,000.00              0.00      150,500.00   
     3     turbine        241,000.00              0.00      162,750.00   
     4     turbine        194,500.00              0.00      156,625.00   
     5     turbine        258,500.00              0.00      926,076.39   
     6     turbine      1,518,500.00              0.00    3,210,986.11   
     7     turbine        210,000.00              0.00      112,966.15   
     8     turbine        160,000.00              0.00       88,703.13   
     9     turbine        134,000.00              0.00      129,992.19   
     10    turbine        257,500.00              0.00      160,125.00   
     11    turbine        126,000.00              0.00      157,500.00   
     12    turbine        178,000.00              0.00      161,875.00   
2005 1     turbine        166,000.00              0.00      161,875.00   
     2     turbine        146,500.00              0.00      144,375.00   
     3     turbine        203,000.00              0.00      161,875.00   
     4     turbine        199,000.00              0.00      143,244.79   

                       total_cost  
year month component               
2003 1     turbine      67,661.46  
     2     turbine      88,510.42  
     3     turbine      97,677.08  
     4     turbine     107,916.67  
     5     turbine     113,377.60  
     6     turbine      95,750.00  
     7     turbine     119,921.88  
     8     turbine      62,869.79  
     9     turbine      65,338.54  
     10    turbine     239,453.13  
     11    turbine   1,225,273.44  
     12    turbine   6,214,343.75  
2004 1     turbine     302,265.63  
     2     turbine     387,500.00  
     3     turbine     403,750.00  
     4     turbine     351,125.00  
     5     turbine   1,184,576.39  
     6     turbine   4,729,486.11  
     7     turbine     322,966.15  
     8     turbine     248,703.13  
     9     turbine     263,992.19  
     10    turbine     417,625.00  
     11    turbine     283,500.00  
     12    turbine     339,875.00  
2005 1     turbine     327,875.00  
     2     turbine     290,875.00  
     3     turbine     364,875.00  
     4     turbine     342,244.79

<a id="fixed-costs"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [44]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [45]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [46]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [47]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"254,844.10","77,722.25","69,583.75",540.00
major repair,"90,610.75","1,900.24","1,364.04",32.00
major replacement,"173,466.86","10,852.19","173,466.86",58.00
manual reset,"218,244.66","37,970.25","13,819.25","5,681.00"
medium repair,"13,106.34","10,120.25","8,015.75",207.00
minor repair,"74,140.74","30,903.75","14,347.50","2,258.00"


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [48]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,624,500,220.50"


In [49]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,624,500,220.50","122,091,785.50","123,453,732.50","118,932,475.00","121,508,635.50","123,512,532.00","123,269,034.50","120,142,016.50","123,302,375.00","119,966,903.00","122,191,256.50","109,949,533.50","121,201,487.50","113,151,725.50","119,035,066.50","119,762,319.50","116,595,319.00","122,945,688.00","123,176,450.00","108,993,702.00","116,375,194.00","122,564,501.50","122,626,768.50","123,227,893.50","122,962,186.00","114,639,651.50","121,941,337.00","116,373,029.00","121,759,623.50","122,567,613.00","117,677,417.50","110,539,300.00","122,952,042.00","123,061,524.50","116,269,152.00","123,996,808.50","122,257,187.00","106,658,980.00","120,400,916.00","120,586,872.00","122,882,199.50","120,011,248.00","123,552,077.50","115,543,756.00","122,439,714.50","121,642,868.50","122,966,042.00","114,534,931.00","122,941,742.50","118,945,605.50","115,417,998.00","122,849,998.00","122,698,297.50","123,318,732.50","122,593,065.00","116,169,145.00","122,483,891.50","122,053,581.00","122,637,270.50","122,932,324.50","122,460,165.50","122,625,199.50","116,872,522.50","122,261,734.00","123,016,700.50","122,530,557.00","117,151,373.50","116,480,740.50","123,510,558.50","123,072,984.00","122,808,981.00","116,943,275.00","121,565,370.00","123,810,296.50","120,985,553.50","122,769,108.00","116,431,601.50","120,938,316.00","123,446,066.50","122,615,993.00","120,968,601.00"


In [50]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"852,889,462.00"
2004,"939,431,944.50"
2005,"969,670,396.50"
2006,"945,199,945.50"
2007,"1,007,444,847.00"
2008,"1,070,642,760.50"
2009,"958,122,431.00"
2010,"861,738,954.50"
2011,"1,026,657,875.50"


In [51]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,077,041,864.50"
2,"820,539,141.50"
3,"900,337,361.50"
4,"708,473,147.00"
5,"683,332,596.50"
6,"556,270,447.00"
7,"589,927,084.00"
8,"639,549,501.50"
9,"786,896,054.00"


In [52]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     112,307,643.50
     2      59,315,350.00
     3      67,404,750.50
     4      91,926,235.00
     5      60,884,437.00
     6      52,629,612.00
     7      52,501,530.50
     8      52,430,787.50
     9      46,711,473.50
     10     72,163,221.00
     11     89,425,901.50
     12     95,188,520.00
2004 1      96,220,771.00
     2      84,428,249.50
     3      89,170,838.00
     4      66,032,505.50
     5      50,850,386.50
     6      65,642,071.00
     7      49,085,498.50
     8      73,021,623.50
     9      99,338,251.50
     10     98,350,140.00
     11     81,192,878.00
     12     86,098,731.50

<a id="pysam"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [53]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [54]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [55]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [56]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [57]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
